In [1]:
import pandas as pd
df = pd.read_excel(r'NewNewNew.xlsx')
df

,公开(公告)号,[标]原始申请(专利权)人,原始申请(专利权)人地址,申请日,IPC主分类号,类别,Year
0,US62875872P0,拜爾拉吉克股份有限公司,US,2019,-,未分类,2
1,US62872589P0,拜爾拉吉克股份有限公司,US,2019,-,未分类,2
2,US675631A,FARBENFABEN OF ELBERFELD,-,1900,C07C,聚合物及其制造工艺,1
3,US20230250275A1,ALGIX,US,2023,C08L,聚合物及其制造工艺,3
4,US20230212537A1,CARBIOS,FR,2022,C12N,微生物或酶、其组合物,3
...,...,...,...,...,...,...,...
445,EP2294125A2,罗盖特公司,FR,2009,C08L,聚合物及其制造工艺,1
446,EP2247661A2,罗盖特公司,FR,2009,C08L,聚合物及其制造工艺,1
447,EP2144221B1,株式会社LG化学,KR,2009,G09F,未分类,1
448,EP2144966A1,法国阿科玛公司,FR,2008,C08L,聚合物及其制造工艺,1


In [121]:
import pandas as pd
import numpy as np

# 将申请日列转换为datetime格式，并提取年份作为新的一列

# 按照年份和IPC主分类号分组，统计每个年份和IPC主分类号下的专利数量
grouped = df.groupby(['Year', 'IPC主分类号'])['IPC主分类号'].count().reset_index(name='count')

# 根据专利数量将数据框分组，并计算每组中的年份数量
bins = [-1, 0, 2, 6, 12, 16, np.inf]
levels = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']
grouped['label'] = pd.cut(grouped['count'], bins=bins, labels=levels )
result = grouped.groupby(['IPC主分类号', 'Year'])['label'].first().unstack(fill_value='L1')
result = result.transpose()
result

IPC主分类号,-,A01G,A01N,A24D,A61K,A61L,A62B,A63H,B01D,B01J,...,D01D,D01F,D02G,D03D,D04H,D21H,F42B,G06F,G09F,H02J
Year,,,,,,,,,,,,,,,,,,,,,
1,L1,L1,L1,L2,L2,L2,L1,L1,L1,L1,...,L1,L1,L1,L1,L2,L2,L1,L2,L2,L1
2,L2,L2,L1,L2,L3,L1,L1,L2,L1,L2,...,L1,L3,L2,L2,L1,L3,L2,L2,L2,L2
3,L1,L2,L2,L2,L3,L1,L2,L1,L2,L1,...,L2,L3,L1,L2,L2,L2,L1,L1,L1,L2


In [122]:
result.to_excel('result.xlsx')

In [123]:
import pandas as pd
df = pd.read_excel(r'result.xlsx')
df

,Year,-,A01G,A01N,A24D,A61K,A61L,A62B,A63H,B01D,...,D01D,D01F,D02G,D03D,D04H,D21H,F42B,G06F,G09F,H02J
0,1,L1,L1,L1,L2,L2,L2,L1,L1,L1,...,L1,L1,L1,L1,L2,L2,L1,L2,L2,L1
1,2,L2,L2,L1,L2,L3,L1,L1,L2,L1,...,L1,L3,L2,L2,L1,L3,L2,L2,L2,L2
2,3,L1,L2,L2,L2,L3,L1,L2,L1,L2,...,L2,L3,L1,L2,L2,L2,L1,L1,L1,L2


In [124]:
# 将year列转换为datetime格式，并提取年份作为新的一列
LF = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']
LR = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']

In [125]:
# 创建转移矩阵
transition_matrix = np.zeros((len(levels), len(levels)))

In [126]:
import numpy as np
# 计算转移概率
def compute_transition_matrix(t, T, LF, LR):
    transition_matrix = np.zeros((len(LF), len(LR)))
    for i in range(len(LF)):
        for j in range(len(LR)):
            ma = 0
            columns = []
            for column in df.columns:
                if column != 'Year':
                    if (df[df['Year'] == t][column] == LF[i]).any():
                        ma += 1
                        columns.append(column)
            mab = 0
            for column in columns:
                mab += (df[df['Year'] == T][column] == LR[j]).sum()
            if ma == 0:
                Mab = 0
            else:
                Mab = mab / ma
            transition_matrix[i, j] = Mab
    return transition_matrix

In [127]:
# 计算转移矩阵
t = 1
T = 2
transition_matrix = compute_transition_matrix(t, T, LF, LR)

In [128]:
Markov_matrix = pd.DataFrame(transition_matrix, index=levels, columns=levels)
Markov_matrix

,L1,L2,L3,L4,L5,L6
L1,0.250000,0.678571,0.035714,0.035714,0.0,0.000000
L2,0.214286,0.285714,0.285714,0.142857,0.0,0.071429
L3,0.000000,0.000000,0.000000,0.333333,0.0,0.666667
L4,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
L5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
L6,0.000000,0.000000,0.000000,0.000000,0.0,1.000000


In [129]:
Markov_matrix.to_csv('Markov_matrix.csv')

In [130]:
import pandas as pd

def identify_transitions(df, year1, year2):

    df_year1 = df[df['Year'] == year1].iloc[0]
    df_year2 = df[df['Year'] == year2].iloc[0]

    positive_transitions = []
    negative_transitions = []

    levels = {'L1': 1, 'L2': 2, 'L3': 3, 'L4': 4}


    for column in df.columns[1:]:
        level_year1 = levels[df_year1[column]]
        level_year2 = levels[df_year2[column]]

        if level_year1 < level_year2:
            positive_transitions.append(f"{column}: L{level_year1} -> L{level_year2}")
        elif level_year1 > level_year2:
            negative_transitions.append(f"{column}: L{level_year1} -> L{level_year2}")

    return positive_transitions, negative_transitions

data = pd.read_excel('result.xlsx')

positive, negative = identify_transitions(data, 1, 2)

print("正向转移:", positive)
print("负向转移:", negative)

KeyError: 'L6'